# Sanity Checker 3

#### utilizes the classes found in `PatchAnalyzer/utils/ephys_analyzer.py` and `PatchAnalyzer/model/data_loader.py` to calculate the following parameters:
* 1️⃣ Passive properties from VC (optional) & CC sweeps  
* 2️⃣ Spike detection → F–I curve & per-spike metrics  
* 3️⃣ One-click plot toggles for debugging  
* Copy-paste-ready cells; all user inputs live in the next “Config” cell


In [ ]:
# ---------------------------------------------------------------------
# 🛠️  USER CONFIG  – fill / tweak these and rerun the rest of the nb
# ---------------------------------------------------------------------


# 0. IMPORT NECCESSARY LIBRARYS ------------------------------------------------
from pathlib import Path
import sys, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# 1. SET PATH FOR EXPERIMENT FOLDER -------------------------------------------
# EXP_DIR = Path(r"C:\Users\sa-forest\Documents\GitHub\PatchAnalyzer\Data\Test\2025_07_02-13_09")
EXP_DIR = Path(r"C:\Users\sa-forest\Documents\GitHub\PatchAnalyzer\Data\Rowan_GFP_TAU_exp\2025_06_29-15_14")


# 2. SELECT A SET OF PROTOCOLS USING CELL_ID -----------------------------------
CELL_ID = 3          # int

# 3. FLAGS TO DETERMINE VISUALIZATION AND VERBOSE RESULTS ----------------------
VERBOSE             = True   # tables & plots
NORMALIZE_BY_CM     = True   # add pA / pF columns / axes
SHOW_RAW_PLOTS      = True
SHOW_PASSIVE_PLOTS  = True
SHOW_FI_PLOTS       = True
SHOW_SPIKE_PLOTS    = True

# --- repo on sys.path + misc --------------------------------------------------
if not EXP_DIR.exists():
    raise RuntimeError(f"EXP_DIR does not exist: {EXP_DIR}")
repo_root = EXP_DIR.parents[2]
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

plt.rcParams["figure.dpi"] = 110
warnings.filterwarnings("ignore", category=RuntimeWarning)





In [ ]:
# ---------------------------------------------------------------------
# 🔄  LOAD METADATA + ALL TRACES for the chosen experiment
# ---------------------------------------------------------------------
from PatchAnalyzer.models.data_loader import (
    load_metadata,
    load_voltage_traces_for_indices,
    load_current_sweeps,
)
from PatchAnalyzer.utils.ephys_analyzer import VprotAnalyzer, CprotAnalyzer
import warnings, matplotlib.pyplot as plt

# 0. LOAD VOLTAGE DATA FOR GIVEN CELL_ID ---------------------------------------
meta_df  = load_metadata([EXP_DIR])
idx_col  = "indices" if "indices" in meta_df.columns else "index"
vp_idxs  = meta_df[idx_col].tolist()
vc_traces = load_voltage_traces_for_indices(EXP_DIR, vp_idxs)

# 1. LOAD CURRENT DATA FOR GIVEN CELL_ID ---------------------------------------
cc_dict  = load_current_sweeps(EXP_DIR, [CELL_ID])
if CELL_ID not in cc_dict:
    raise RuntimeError(f"CELL_ID {CELL_ID} not found in {EXP_DIR}")
cc_sweeps = cc_dict[CELL_ID]

# 2. BUILD ANALYZER OBJECTS ----------------------------------------------------
v_an = VprotAnalyzer(debug=False)
c_an = CprotAnalyzer(debug=False)

# ---- diagnostic raw-trace plot ----------------------------------------------
if VERBOSE and SHOW_RAW_PLOTS and cc_sweeps:
    sel = [0, len(cc_sweeps)//2, len(cc_sweeps)-1] if len(cc_sweeps) > 2 else range(len(cc_sweeps))
    fig, axs = plt.subplots(len(sel), 2, figsize=(7, 2*len(sel)), sharex='col')
    if len(sel) == 1:
        axs = [axs]                   # always iterable
    for ax_pair, idx in zip(axs, sel):
        sw  = cc_sweeps[idx]
        t   = sw.time
        cmd = sw.command_pA
        rsp = sw.response_mV
        ts, te = c_an._find_test_pulse(cmd)
        es, ee = c_an._find_experimental_pulse(cmd, te)
        ax_pair[0].plot(t, cmd); ax_pair[0].set_ylabel("Cmd (pA)")
        ax_pair[1].plot(t, rsp); ax_pair[1].set_ylabel("Rsp (mV)")
        if ts is not None:
            for a in ax_pair: a.axvspan(t[ts], t[te], color='orange', alpha=.25)
        if es is not None:
            for a in ax_pair: a.axvspan(t[es], t[ee], color='skyblue', alpha=.25)
    axs[-1][0].set_xlabel("Time (s)")
    axs[-1][1].set_xlabel("Time (s)")
    plt.suptitle("Representative raw sweeps (orange =test, blue = experiment)")
    plt.tight_layout(); plt.show()


In [ ]:
# -----------------------------------------------------------------------------------------
# 🔍  CURRENT-CLAMP and VOLTAGE-CLAMP passive parameters – per sweep
# -----------------------------------------------------------------------------------------
import numpy as np, pandas as pd, matplotlib.pyplot as plt

# 1.A  VOLTAGE-CLAMP passives ---------------------------------------------------
Ra_MΩ, Rm_MΩ, Cm_pF = v_an.fit_cell(vc_traces, aggregate="mean")
Ra_Ω = float(Ra_MΩ)*1e6
Rm_Ω = float(Rm_MΩ)*1e6
R_para = 1 / (1 /Ra_Ω + 1 / Rm_Ω)
Cm_F = float(Cm_pF)*1e-12
tau_s = Cm_F*R_para
tau_ms = tau_s*1e3
if VERBOSE:
    print(f"Voltage-Clamp passive (mean):  Ra={Ra_MΩ:.1f} MΩ   Rm={Rm_MΩ:.1f} MΩ   Cm={Cm_pF:.1f} pF  Tau={tau_ms:.1f} ms")

# 2.A/B  CURRENT-CLAMP per-sweep passives --------------------------------------
passive_rows = []
for s_idx, sw in enumerate(cc_sweeps, 1):
    try:
        res = c_an.passive_params(
            sw.time,
            sw.command_pA / c_an.cclamp_gain,   # pA → V
            sw.response_mV / c_an.vclamp_gain, # mV → V
            return_intermediates=False,
        )
        res["sweep"] = s_idx
        passive_rows.append(res)
    except RuntimeError as exc:
        warnings.warn(f"Sweep {s_idx}: {exc}")

cc_passive_df = pd.DataFrame(passive_rows).set_index("sweep")
if VERBOSE:
    display(cc_passive_df)

# ---- box-and-whisker with VC overlay & robust outlier handling ----------
props = ["input_resistance_MOhm",
         "membrane_tau_ms",
         "membrane_capacitance_pF",
         "resting_potential_mV"]

clean_vals, outliers = {}, {}
for p in props:
    vals = cc_passive_df[p].dropna()

    # ------- robust filter: Median ± 3 × MAD -----------------------------
    med = np.median(vals)
    mad = np.median(np.abs(vals - med)) or 1e-12      # avoid zero-mad
    thresh = 5 * 1.4826 * mad                         # MAD → SD-like
    keep = vals[np.abs(vals - med) <= thresh]
    out  = vals[np.abs(vals - med) >  thresh]

    clean_vals[p], outliers[p] = keep, out

if VERBOSE and SHOW_PASSIVE_PLOTS:
    fig, axs = plt.subplots(1, len(props), figsize=(3.5*len(props), 3))
    for ax, p in zip(axs, props):
        ax.boxplot(
            clean_vals[p],
            widths=.6,
            whis=[0, 100],              # whiskers span full range
            showfliers=False,           # no Matplotlib “fliers”
            patch_artist=True,
            boxprops   =dict(facecolor='none', edgecolor='black'),
            medianprops=dict(color='black'),
            whiskerprops=dict(color='black'),
            capprops   =dict(color='black'),
        )
        ax.scatter(np.random.normal(1, 0.04, len(clean_vals[p])),
                   clean_vals[p], s=35, color='#003f5c', zorder=3)

        # VC overlay (red circle) ---------------------------------------
        overlay = {"input_resistance_MOhm": Rm_MΩ,
                "membrane_tau_ms":       tau_ms,   # ← NEW
                "membrane_capacitance_pF": Cm_pF}.get(p, np.nan)
        if not np.isnan(overlay):
            ax.scatter([1], [overlay], c='#8b0000', marker='o',
                       s=70, zorder=4, label='VC mean')

        ax.set_title(p.replace("_", "\n"))
        ax.grid(True, linestyle='--', alpha=.3)
        if p == "resting_potential_mV":
            ax.set_ylabel("mV")
        else:
            ax.ticklabel_format(axis='y', style='plain')

        # mean value written underneath axis ----------------------------
        mu = clean_vals[p].mean()
        ax.text(1, ax.get_ylim()[0] - 0.07*np.ptp(ax.get_ylim()),
                f"μ={mu:.2f}", ha='center', va='top', fontsize=9)

    axs[0].legend(loc="best")
    plt.suptitle("CC passive parameters (red = VC mean)")
    plt.tight_layout(); plt.show()

# ---- recompute mean Cm (from cleaned values) ----------------------------
Cm_mean_pF = clean_vals["membrane_capacitance_pF"].mean()


In [ ]:
# ---------------------------------------------------------------------
# ⚡  SPIKE DETECTION  &  F–I CURVE
# ---------------------------------------------------------------------
sweep_tuples = [
    (sw.time,
     sw.command_pA / c_an.cclamp_gain,    # back-convert to Volts
     sw.response_mV / c_an.vclamp_gain)   # back-convert to Volts
    for sw in cc_sweeps
]

fi_df = c_an.firing_curve(
    sweep_tuples,
    normalize_by_Cm=NORMALIZE_BY_CM,
    Cm_pF=Cm_mean_pF if NORMALIZE_BY_CM else None,
)
# --- auto-fix if values ended up in A/pF (≤1e-6) -----------------------
if NORMALIZE_BY_CM and fi_df["current_inj_pApF"].abs().max() < 1e-6:
    fi_df["current_inj_pApF"] *= 1e12   # A➜pA

if VERBOSE: display(fi_df)

if VERBOSE and SHOW_FI_PLOTS and not fi_df.empty:
    x = "current_inj_pApF" if NORMALIZE_BY_CM else "current_inj_pA"
    plt.figure(figsize=(4,3))
    plt.plot(fi_df[x], fi_df["mean_firing_frequency_Hz"], 'o-',
             color='#003f5c', markerfacecolor='#003f5c')
    plt.xlabel("Injected current (pA/pF)" if NORMALIZE_BY_CM else "Injected current (pA)")
    plt.ylabel("Mean firing freq (Hz)")
    plt.title("F–I curve")
    plt.grid(True, linestyle='--', alpha=.35)
    plt.tight_layout(); plt.show()



In [ ]:
# ---------------------------------------------------------------------
# 🔬  PER-SPIKE METRICS  (half-width, AHP, threshold, dV/dt-max)
# ---------------------------------------------------------------------
spk_df = c_an.spike_metrics(
    sweep_tuples,
    normalize_by_Cm=NORMALIZE_BY_CM,
    Cm_pF=Cm_mean_pF if NORMALIZE_BY_CM else None,
)
# --- auto-fix unit if needed ------------------------------------------
if NORMALIZE_BY_CM and "current_inj_pApF" in spk_df.columns:
    if spk_df["current_inj_pApF"].abs().max() < 1e-6:
        spk_df["current_inj_pApF"] *= 1e12

if VERBOSE: display(spk_df.head())

if VERBOSE and SHOW_SPIKE_PLOTS and not spk_df.empty:
    metrics = ["half_width_ms", "AHP_mV", "dvdt_max_mV_per_ms"]
    xkey = "current_inj_pApF" if NORMALIZE_BY_CM else "current_inj_pA"

    grp = spk_df.groupby(xkey)               # aggregate by injected current
    fig, axs = plt.subplots(1, len(metrics), figsize=(4*len(metrics), 3))

    for ax, m in zip(axs, metrics):
        means = grp[m].mean()
        ax.plot(means.index, means.values, 'o-', color='#003f5c')
        ax.set_xlabel("I (pA/pF)" if NORMALIZE_BY_CM else "I (pA)")
        ax.set_ylabel(m)
        ax.set_title(m)
        ax.grid(True, linestyle='--', alpha=.35)

    plt.suptitle("Mean per-sweep spike metrics vs injected current")
    plt.tight_layout(); plt.show()
\

